### Database with 6*6 images

shapes : square, triangle, cirle
colors : light red, light blue, light yellow, light green

dataset size : 5000

probability to have a symbol at each position : 0.5

Small database to develop counterfactual generation feature


In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/dblocaltest0.1/"
os.makedirs(db_dir, exist_ok=True)

In [3]:
from xaipatimg.datagen.gendataset import generic_rule_exist_row_with_only_shape

rules_data = [
    {"name": "simple_1_row_circles", "gen_fun": generic_rule_exist_row_with_only_shape, "gen_kwargs": {"shape": "circle"}}
]

In [4]:
# Number of images generated
NBGEN = 1000

# Grid division of each image
X_DIVISIONS = 6
Y_DIVISIONS = 6

# Size of the images in pixels
img_size = (700, 700)

# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [5]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [6]:
import numpy as np
from xaipatimg.datagen.dbimg import generate_uuid
import os

to_generate = NBGEN
unique_content_generated = {}
duplicate_count = 0
while to_generate > 0:
    content = []
    for i in range(X_DIVISIONS):
        for j in range(Y_DIVISIONS):
            if np.random.random() < SHAPE_PROB:
                content.append({
                    "shape": np.random.choice(SHAPES),
                    "pos": (i, j),
                    "color": np.random.choice(COLORS)
                })

    if str(content) in unique_content_generated:
        duplicate_count += 1
        continue

    imgid = generate_uuid()
    db[imgid] = {
        "path": os.path.join("img", imgid + ".png"),
        "division" : (X_DIVISIONS, Y_DIVISIONS),
        "size": img_size,
        "content": content
    }

    unique_content_generated[str(content)] = True
    to_generate -= 1

print("unique generated in DB : " + str(len(db)))
print("duplicates avoided : " + str(duplicate_count))

unique generated in DB : 3000
duplicates avoided : 0


In [ ]:
import tqdm

content_dict = {}
nb_duplicates = 0

for k, v in tqdm.tqdm(db.items()):
    if str(v["content"]) in content_dict:
        nb_duplicates += 1
    else:
        content_dict[str(v["content"])] = True

print(nb_duplicates)

In [ ]:
from xaipatimg.datagen.genimg import gen_img_and_save_db
# gen_img_and_save_db(db, db_dir, overwrite=False, n_jobs=20)

In [ ]:
from xaipatimg.datagen.gendataset import create_dataset_generic_rule_extract_sample

for rule_d in rules_data:
    create_dataset_generic_rule_extract_sample(d)